In [ ]:
pip install anthropic pydantic 'pydantic[email]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.3 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
CLAUDE_API_KEY = userdata.get('CLAUDE_API_KEY')

In [ ]:
from anthropic import Anthropic
from pydantic import BaseModel, EmailStr, Field
from typing import Optional

client = Anthropic(api_key=CLAUDE_API_KEY)
MODEL_NAME = "claude-sonnet-4-20250514"

In [ ]:
class Author(BaseModel):
    name: str
    email: EmailStr

class Note(BaseModel):
    note: str
    author: Author
    tags: Optional[list[str]] = None
    priority: int = Field(ge=1, le=5, default=3)
    is_public: bool = False

class SaveNoteResponse(BaseModel):
    success: bool
    message: str

In [ ]:
tools = [
    {
        "name": "save_note",
        "description": "A tool that saves a note with the author and metadata.",
        "input_schema": {
            "type": "object",
            "properties": {
                "note": {
                    "type": "string",
                    "description": "The content of the note to be saved."
                },
                "author": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name of the author."
                        },
                        "email": {
                            "type": "string",
                            "format": "email",
                            "description": "The email address of the author."
                        }
                    },
                    "required": ["name", "email"]
                },
                "priority": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": 5,
                    "default": 3,
                    "description": "The priority level of the note (1-5)."
                },
                "is_public": {
                    "type": "boolean",
                    "default": False,
                    "description": "Indicates whether the note is publicly accessible."
                }
            },
            "required": ["note", "author"]
        }
    }
]

In [ ]:
def save_note(note: str, author: dict, priority: int = 3, is_public: bool = False) -> None:
    print("Note saved successfully!")

In [ ]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "save_note":
        note = Note(
            note=tool_input["note"],
            author=Author(
                name=tool_input["author"]["name"],
                email=tool_input["author"]["email"]
            ),
            priority=tool_input.get("priority", 3),
            is_public=tool_input.get("is_public", False)
        )
        save_note(note.note, note.author.model_dump(), note.priority, note.is_public)
        return SaveNoteResponse(success=True, message="Note saved successfully!")

def generate_response(save_note_response):
    return f"Response: {save_note_response.message}"

In [ ]:
def chatbot_interaction(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    messages = [
        {"role": "user", "content": user_message}
    ]

    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        messages=messages
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content: {message.content}")

    if message.stop_reason == "tool_use":
        tool_use = next(block for block in message.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input: {tool_input}")

        save_note_response = process_tool_call(tool_name, tool_input)


        print(f"Tool Result: {save_note_response}")

        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            messages=[
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": message.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": str(save_note_response),
                        }
                    ],
                },
            ],
            tools=tools,
        )
    else:
        response = message

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )
    print(response.content)
    print(f"\nFinal Response: {final_response}")

    return final_response

In [ ]:
chatbot_interaction("""
Can you save a private note with the following details?
Note: Remember to buy milk and eggs.
Author: John Doe (johndoe@gmail.com)
Priority: 4
""")


User Message: 
Can you save a private note with the following details?
Note: Remember to buy milk and eggs.
Author: John Doe (johndoe@gmail.com)
Priority: 4


Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll save that private note for you with the specified details.", type='text'), ToolUseBlock(id='toolu_01ASqH5gJuoFqZb8capEjgQy', input={'note': 'Remember to buy milk and eggs.', 'author': {'name': 'John Doe', 'email': 'johndoe@gmail.com'}, 'priority': 4, 'is_public': False}, name='save_note', type='tool_use')]

Tool Used: save_note
Tool Input: {'note': 'Remember to buy milk and eggs.', 'author': {'name': 'John Doe', 'email': 'johndoe@gmail.com'}, 'priority': 4, 'is_public': False}
Note saved successfully!
Tool Result: success=True message='Note saved successfully!'
[TextBlock(citations=None, text='Perfect! I\'ve successfully saved your private note with the following details:\n- Note: "Remember to buy milk and eggs."\n- Author: John Doe (johndoe@g

'Perfect! I\'ve successfully saved your private note with the following details:\n- Note: "Remember to buy milk and eggs."\n- Author: John Doe (johndoe@gmail.com)\n- Priority: 4\n- Privacy: Private (not publicly accessible)\n\nThe note has been saved successfully!'